<a href="https://colab.research.google.com/github/WakWakBird/MLS/blob/main/TensorFlow_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x


# 새 섹션

In [12]:
# Lab 5: Logistic Regression Classifier for Diabetes Dataset
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 1. 데이터 로드: 8개의 입력 특성 + 1개의 이진 정답
xy = np.loadtxt('../data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]         # 입력 데이터 (shape: [n_samples, 8])
y_data = xy[:, [-1]]         # 출력 라벨 (shape: [n_samples, 1], 2D 유지)

# 데이터 확인
print("x_data shape:", x_data.shape)
print("y_data shape:", y_data.shape)

# 2. 모델 구성
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=1, input_dim=x_data.shape[1], activation='sigmoid'))  # 출력 1개 (0~1 확률)

# 3. 모델 컴파일
model.compile(
    loss='binary_crossentropy',                             # 이진 분류에 적합한 손실 함수
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),  # SGD 옵티마이저
    metrics=['accuracy']                                    # 정확도 측정
)

# 4. 모델 구조 요약 출력
model.summary()

# 5. 모델 학습
history = model.fit(x_data, y_data, epochs=500)

# 6. 학습 정확도 출력
print("\nTraining Accuracy: {:.4f}".format(history.history['accuracy'][-1]))

# 7. 개별 샘플 예측
sample = [[0.176471, 0.155779, 0, 0, 0, 0.052161, -0.952178, -0.733333]]  # 정규화된 데이터
y_predict = model.predict(sample)
print("Sample prediction (확률):", y_predict)
print("Sample prediction (클래스):", int(y_predict[0][0] > 0.5))  # 0.5 기준 분류

# 8. 전체 데이터로 성능 평가
loss, acc = model.evaluate(x_data, y_data)
print("\nEvaluate result → Loss: {:.4f}, Accuracy: {:.4f}".format(loss, acc))

# 9. 학습 그래프 시각화 (손실 & 정확도)
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)

plt.tight_layout()
plt.show()


FileNotFoundError: ../data-03-diabetes.csv not found.